# Enhanced Checkpoint System

This notebook implements a comprehensive checkpoint system with auto-resume capabilities for training and HPO.
It saves complete training state including model, optimizer, scheduler, and metadata.

In [ ]:
import os
import json
import pickle
import shutil
import torch
import optuna
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## Checkpoint Data Structures

Define comprehensive data structures for storing training state.

In [ ]:
@dataclass
class TrainingMetrics:
    """Training metrics for a specific epoch."""
    epoch: int
    train_loss: float
    val_loss: float
    val_metrics: Dict[str, float]
    learning_rate: float
    timestamp: str
    
@dataclass
class CheckpointMetadata:
    """Metadata for a checkpoint."""
    checkpoint_id: str
    experiment_name: str
    model_type: str
    created_at: str
    epoch: int
    step: int
    best_metric: float
    best_epoch: int
    total_epochs: int
    config_hash: str
    git_commit: Optional[str] = None
    notes: str = ""
    
@dataclass
class TrainingState:
    """Complete training state."""
    epoch: int
    step: int
    best_metric: float
    best_epoch: int
    epochs_without_improve: int
    training_history: List[TrainingMetrics]
    random_state: Dict[str, Any]
    scaler_state: Optional[Dict[str, Any]] = None
    ema_state: Optional[Dict[str, Any]] = None

@dataclass
class HPOState:
    """HPO study state."""
    study_name: str
    n_trials_completed: int
    best_trial: Optional[Dict[str, Any]]
    best_value: Optional[float]
    trials_history: List[Dict[str, Any]]
    created_at: str
    last_updated: str

## Enhanced Checkpoint Manager

A comprehensive checkpoint manager with versioning and metadata tracking.

In [ ]:
class EnhancedCheckpointManager:
    """Enhanced checkpoint manager with comprehensive state saving and auto-resume."""
    
    def __init__(self, checkpoint_dir: str = "./checkpoints", max_checkpoints: int = 5):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.max_checkpoints = max_checkpoints
        self.metadata_file = self.checkpoint_dir / "metadata.json"
        self.training_log = self.checkpoint_dir / "training_log.json"
        
        # Initialize metadata if not exists
        if not self.metadata_file.exists():
            self._save_metadata({})
    
    def _generate_checkpoint_id(self) -> str:
        """Generate unique checkpoint ID."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        return f"checkpoint_{timestamp}"
    
    def _get_config_hash(self, config: Dict[str, Any]) -> str:
        """Generate hash for configuration to detect changes."""
        import hashlib
        config_str = json.dumps(config, sort_keys=True)
        return hashlib.md5(config_str.encode()).hexdigest()[:8]
    
    def _save_metadata(self, metadata: Dict[str, Any]) -> None:
        """Save checkpoint metadata."""
        with open(self.metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
    
    def _load_metadata(self) -> Dict[str, Any]:
        """Load checkpoint metadata."""
        if self.metadata_file.exists():
            with open(self.metadata_file, 'r') as f:
                return json.load(f)
        return {}
    
    def _cleanup_old_checkpoints(self) -> None:
        """Remove old checkpoints to maintain max_checkpoints limit."""
        metadata = self._load_metadata()
        checkpoints = list(metadata.keys())
        
        if len(checkpoints) > self.max_checkpoints:
            # Sort by creation time and remove oldest
            checkpoints.sort(key=lambda x: metadata[x].get('created_at', ''))
            to_remove = checkpoints[:-self.max_checkpoints]
            
            for checkpoint_id in to_remove:
                checkpoint_path = self.checkpoint_dir / checkpoint_id
                if checkpoint_path.exists():
                    shutil.rmtree(checkpoint_path)
                del metadata[checkpoint_id]
            
            self._save_metadata(metadata)
    
    def save_checkpoint(
        self,
        model: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler: Any,
        training_state: TrainingState,
        config: Dict[str, Any],
        experiment_name: str = "default",
        notes: str = "",
        is_best: bool = False
    ) -> str:
        """Save complete training checkpoint."""
        
        checkpoint_id = self._generate_checkpoint_id()
        checkpoint_path = self.checkpoint_dir / checkpoint_id
        checkpoint_path.mkdir(exist_ok=True)
        
        # Save model state
        torch.save(model.state_dict(), checkpoint_path / "model.pt")
        
        # Save optimizer state
        torch.save(optimizer.state_dict(), checkpoint_path / "optimizer.pt")
        
        # Save scheduler state
        if hasattr(scheduler, 'state_dict'):
            torch.save(scheduler.state_dict(), checkpoint_path / "scheduler.pt")
        
        # Save training state
        with open(checkpoint_path / "training_state.json", 'w') as f:
            json.dump(asdict(training_state), f, indent=2, default=str)
        
        # Save configuration
        with open(checkpoint_path / "config.json", 'w') as f:
            json.dump(config, f, indent=2, default=str)
        
        # Create checkpoint metadata
        metadata = CheckpointMetadata(
            checkpoint_id=checkpoint_id,
            experiment_name=experiment_name,
            model_type=config.get('model', {}).get('encoder', {}).get('type', 'unknown'),
            created_at=datetime.now().isoformat(),
            epoch=training_state.epoch,
            step=training_state.step,
            best_metric=training_state.best_metric,
            best_epoch=training_state.best_epoch,
            total_epochs=config.get('training', {}).get('max_epochs', 0),
            config_hash=self._get_config_hash(config),
            notes=notes
        )
        
        # Save metadata
        with open(checkpoint_path / "metadata.json", 'w') as f:
            json.dump(asdict(metadata), f, indent=2)
        
        # Update global metadata
        global_metadata = self._load_metadata()
        global_metadata[checkpoint_id] = asdict(metadata)
        self._save_metadata(global_metadata)
        
        # Create best checkpoint symlink
        if is_best:
            best_link = self.checkpoint_dir / "best_checkpoint"
            if best_link.exists():
                best_link.unlink()
            best_link.symlink_to(checkpoint_id)
        
        # Cleanup old checkpoints
        self._cleanup_old_checkpoints()
        
        print(f"✅ Checkpoint saved: {checkpoint_id}")
        print(f"   Epoch: {training_state.epoch}")
        print(f"   Best metric: {training_state.best_metric:.4f}")
        print(f"   Path: {checkpoint_path}")
        
        return checkpoint_id

In [ ]:
    def load_checkpoint(
        self,
        checkpoint_id: str,
        model: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler: Any,
        device: torch.device
    ) -> Tuple[TrainingState, Dict[str, Any]]:
        """Load complete training checkpoint."""
        
        checkpoint_path = self.checkpoint_dir / checkpoint_id
        if not checkpoint_path.exists():
            raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")
        
        # Load model state
        model_path = checkpoint_path / "model.pt"
        if model_path.exists():
            model.load_state_dict(torch.load(model_path, map_location=device))
            print(f"✅ Loaded model state from {checkpoint_id}")
        
        # Load optimizer state
        optimizer_path = checkpoint_path / "optimizer.pt"
        if optimizer_path.exists():
            optimizer.load_state_dict(torch.load(optimizer_path, map_location=device))
            print(f"✅ Loaded optimizer state from {checkpoint_id}")
        
        # Load scheduler state
        scheduler_path = checkpoint_path / "scheduler.pt"
        if scheduler_path.exists() and hasattr(scheduler, 'load_state_dict'):
            scheduler.load_state_dict(torch.load(scheduler_path, map_location=device))
            print(f"✅ Loaded scheduler state from {checkpoint_id}")
        
        # Load training state
        training_state_path = checkpoint_path / "training_state.json"
        with open(training_state_path, 'r') as f:
            training_state_dict = json.load(f)
        
        # Convert back to TrainingState object
        training_state = TrainingState(**training_state_dict)
        
        # Load configuration
        config_path = checkpoint_path / "config.json"
        with open(config_path, 'r') as f:
            config = json.load(f)
        
        # Restore random states
        if training_state.random_state:
            torch.set_rng_state(torch.tensor(training_state.random_state['torch']))
            np.random.set_state(tuple(training_state.random_state['numpy']))
            if torch.cuda.is_available():
                torch.cuda.set_rng_state(torch.tensor(training_state.random_state['cuda']))
        
        print(f"✅ Loaded training state from {checkpoint_id}")
        print(f"   Resuming from epoch: {training_state.epoch}")
        print(f"   Best metric so far: {training_state.best_metric:.4f}")
        
        return training_state, config
    
    def find_latest_checkpoint(self, experiment_name: Optional[str] = None) -> Optional[str]:
        """Find the latest checkpoint for an experiment."""
        metadata = self._load_metadata()
        
        if not metadata:
            return None
        
        # Filter by experiment name if provided
        if experiment_name:
            filtered_metadata = {
                k: v for k, v in metadata.items() 
                if v.get('experiment_name') == experiment_name
            }
        else:
            filtered_metadata = metadata
        
        if not filtered_metadata:
            return None
        
        # Find latest by creation time
        latest_checkpoint = max(
            filtered_metadata.keys(),
            key=lambda x: filtered_metadata[x].get('created_at', '')
        )
        
        return latest_checkpoint
    
    def find_best_checkpoint(self, experiment_name: Optional[str] = None) -> Optional[str]:
        """Find the best checkpoint for an experiment."""
        metadata = self._load_metadata()
        
        if not metadata:
            return None
        
        # Filter by experiment name if provided
        if experiment_name:
            filtered_metadata = {
                k: v for k, v in metadata.items() 
                if v.get('experiment_name') == experiment_name
            }
        else:
            filtered_metadata = metadata
        
        if not filtered_metadata:
            return None
        
        # Find best by metric
        best_checkpoint = max(
            filtered_metadata.keys(),
            key=lambda x: filtered_metadata[x].get('best_metric', -float('inf'))
        )
        
        return best_checkpoint
    
    def list_checkpoints(self, experiment_name: Optional[str] = None) -> pd.DataFrame:
        """List all checkpoints as a DataFrame."""
        metadata = self._load_metadata()
        
        if not metadata:
            return pd.DataFrame()
        
        # Filter by experiment name if provided
        if experiment_name:
            filtered_metadata = {
                k: v for k, v in metadata.items() 
                if v.get('experiment_name') == experiment_name
            }
        else:
            filtered_metadata = metadata
        
        if not filtered_metadata:
            return pd.DataFrame()
        
        # Convert to DataFrame
        df_data = []
        for checkpoint_id, meta in filtered_metadata.items():
            df_data.append({
                'checkpoint_id': checkpoint_id,
                'experiment': meta.get('experiment_name', ''),
                'model_type': meta.get('model_type', ''),
                'epoch': meta.get('epoch', 0),
                'best_metric': meta.get('best_metric', 0.0),
                'created_at': meta.get('created_at', ''),
                'notes': meta.get('notes', '')
            })
        
        df = pd.DataFrame(df_data)
        if not df.empty:
            df = df.sort_values('created_at', ascending=False)
        
        return df
    
    def delete_checkpoint(self, checkpoint_id: str) -> None:
        """Delete a specific checkpoint."""
        checkpoint_path = self.checkpoint_dir / checkpoint_id
        if checkpoint_path.exists():
            shutil.rmtree(checkpoint_path)
        
        # Update metadata
        metadata = self._load_metadata()
        if checkpoint_id in metadata:
            del metadata[checkpoint_id]
            self._save_metadata(metadata)
        
        print(f"🗑️  Deleted checkpoint: {checkpoint_id}")
    
    def get_checkpoint_info(self, checkpoint_id: str) -> Dict[str, Any]:
        """Get detailed information about a checkpoint."""
        checkpoint_path = self.checkpoint_dir / checkpoint_id
        if not checkpoint_path.exists():
            raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")
        
        # Load metadata
        metadata_path = checkpoint_path / "metadata.json"
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        # Load training state
        training_state_path = checkpoint_path / "training_state.json"
        with open(training_state_path, 'r') as f:
            training_state = json.load(f)
        
        # Calculate checkpoint size
        total_size = sum(f.stat().st_size for f in checkpoint_path.rglob('*') if f.is_file())
        
        return {
            'metadata': metadata,
            'training_state': training_state,
            'size_mb': total_size / (1024 * 1024),
            'files': [f.name for f in checkpoint_path.iterdir() if f.is_file()]
        }

## Auto-Resume Functionality

Automatic detection and resumption of interrupted training sessions.

In [ ]:
class AutoResumeManager:
    """Manages automatic resumption of training and HPO sessions."""
    
    def __init__(self, checkpoint_manager: EnhancedCheckpointManager):
        self.checkpoint_manager = checkpoint_manager
    
    def should_resume_training(
        self, 
        experiment_name: str, 
        config: Dict[str, Any]
    ) -> Tuple[bool, Optional[str]]:
        """Check if training should be resumed."""
        
        latest_checkpoint = self.checkpoint_manager.find_latest_checkpoint(experiment_name)
        
        if not latest_checkpoint:
            return False, None
        
        # Check if configuration matches
        try:
            checkpoint_info = self.checkpoint_manager.get_checkpoint_info(latest_checkpoint)
            checkpoint_config_hash = checkpoint_info['metadata']['config_hash']
            current_config_hash = self.checkpoint_manager._get_config_hash(config)
            
            if checkpoint_config_hash != current_config_hash:
                print(f"⚠️  Configuration changed since last checkpoint")
                print(f"   Checkpoint config hash: {checkpoint_config_hash}")
                print(f"   Current config hash: {current_config_hash}")
                return False, None
            
            # Check if training is already complete
            training_state = checkpoint_info['training_state']
            max_epochs = config.get('training', {}).get('max_epochs', 100)
            
            if training_state['epoch'] >= max_epochs:
                print(f"✅ Training already complete ({training_state['epoch']}/{max_epochs} epochs)")
                return False, None
            
            print(f"🔄 Found resumable checkpoint: {latest_checkpoint}")
            print(f"   Experiment: {experiment_name}")
            print(f"   Epoch: {training_state['epoch']}/{max_epochs}")
            print(f"   Best metric: {training_state['best_metric']:.4f}")
            
            return True, latest_checkpoint
            
        except Exception as e:
            print(f"❌ Error checking checkpoint: {e}")
            return False, None
    
    def resume_training(
        self,
        checkpoint_id: str,
        model: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler: Any,
        device: torch.device
    ) -> Tuple[TrainingState, Dict[str, Any]]:
        """Resume training from checkpoint."""
        
        print(f"🔄 Resuming training from checkpoint: {checkpoint_id}")
        
        training_state, config = self.checkpoint_manager.load_checkpoint(
            checkpoint_id, model, optimizer, scheduler, device
        )
        
        print(f"✅ Training resumed successfully")
        print(f"   Starting from epoch: {training_state.epoch + 1}")
        print(f"   Best metric so far: {training_state.best_metric:.4f}")
        print(f"   Training history: {len(training_state.training_history)} epochs")
        
        return training_state, config
    
    def create_training_state(
        self,
        epoch: int = 0,
        step: int = 0,
        best_metric: float = -float('inf'),
        best_epoch: int = -1
    ) -> TrainingState:
        """Create a new training state for fresh training."""
        
        # Capture random states
        random_state = {
            'torch': torch.get_rng_state().tolist(),
            'numpy': list(np.random.get_state()),
        }
        
        if torch.cuda.is_available():
            random_state['cuda'] = torch.cuda.get_rng_state().tolist()
        
        return TrainingState(
            epoch=epoch,
            step=step,
            best_metric=best_metric,
            best_epoch=best_epoch,
            epochs_without_improve=0,
            training_history=[],
            random_state=random_state
        )

## HPO Checkpoint Manager

Specialized checkpoint management for Optuna HPO studies.

In [ ]:
class HPOCheckpointManager:
    """Manages checkpoints for HPO studies with auto-resume capabilities."""
    
    def __init__(self, hpo_dir: str = "./hpo_checkpoints"):
        self.hpo_dir = Path(hpo_dir)
        self.hpo_dir.mkdir(parents=True, exist_ok=True)
    
    def save_hpo_state(
        self,
        study: optuna.Study,
        config: Dict[str, Any],
        notes: str = ""
    ) -> None:
        """Save HPO study state."""
        
        study_name = study.study_name
        study_dir = self.hpo_dir / study_name
        study_dir.mkdir(exist_ok=True)
        
        # Get study statistics
        trials = study.trials
        best_trial = study.best_trial if trials else None
        best_value = study.best_value if trials else None
        
        # Create HPO state
        hpo_state = HPOState(
            study_name=study_name,
            n_trials_completed=len(trials),
            best_trial=best_trial.__dict__ if best_trial else None,
            best_value=best_value,
            trials_history=[trial.__dict__ for trial in trials],
            created_at=datetime.now().isoformat(),
            last_updated=datetime.now().isoformat()
        )
        
        # Save HPO state
        with open(study_dir / "hpo_state.json", 'w') as f:
            json.dump(asdict(hpo_state), f, indent=2, default=str)
        
        # Save configuration
        with open(study_dir / "config.json", 'w') as f:
            json.dump(config, f, indent=2, default=str)
        
        # Save study object (pickle)
        with open(study_dir / "study.pkl", 'wb') as f:
            pickle.dump(study, f)
        
        print(f"💾 HPO state saved for study: {study_name}")
        print(f"   Trials completed: {len(trials)}")
        if best_value is not None:
            print(f"   Best value: {best_value:.4f}")
    
    def load_hpo_state(self, study_name: str) -> Tuple[HPOState, Dict[str, Any]]:
        """Load HPO study state."""
        
        study_dir = self.hpo_dir / study_name
        if not study_dir.exists():
            raise FileNotFoundError(f"HPO state not found for study: {study_name}")
        
        # Load HPO state
        with open(study_dir / "hpo_state.json", 'r') as f:
            hpo_state_dict = json.load(f)
        
        hpo_state = HPOState(**hpo_state_dict)
        
        # Load configuration
        with open(study_dir / "config.json", 'r') as f:
            config = json.load(f)
        
        return hpo_state, config
    
    def should_resume_hpo(self, study_name: str, config: Dict[str, Any]) -> bool:
        """Check if HPO study should be resumed."""
        
        study_dir = self.hpo_dir / study_name
        if not study_dir.exists():
            return False
        
        try:
            hpo_state, saved_config = self.load_hpo_state(study_name)
            
            # Check if study is complete
            max_trials = config.get('hpo', {}).get('n_trials', 100)
            if hpo_state.n_trials_completed >= max_trials:
                print(f"✅ HPO study already complete ({hpo_state.n_trials_completed}/{max_trials} trials)")
                return False
            
            print(f"🔄 Found resumable HPO study: {study_name}")
            print(f"   Trials completed: {hpo_state.n_trials_completed}/{max_trials}")
            if hpo_state.best_value is not None:
                print(f"   Best value so far: {hpo_state.best_value:.4f}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error checking HPO state: {e}")
            return False
    
    def list_hpo_studies(self) -> pd.DataFrame:
        """List all HPO studies."""
        
        studies = []
        for study_dir in self.hpo_dir.iterdir():
            if study_dir.is_dir():
                try:
                    hpo_state, _ = self.load_hpo_state(study_dir.name)
                    studies.append({
                        'study_name': hpo_state.study_name,
                        'n_trials': hpo_state.n_trials_completed,
                        'best_value': hpo_state.best_value,
                        'last_updated': hpo_state.last_updated
                    })
                except Exception:
                    continue
        
        if studies:
            df = pd.DataFrame(studies)
            df = df.sort_values('last_updated', ascending=False)
            return df
        else:
            return pd.DataFrame()
    
    def delete_hpo_study(self, study_name: str) -> None:
        """Delete HPO study data."""
        study_dir = self.hpo_dir / study_name
        if study_dir.exists():
            shutil.rmtree(study_dir)
            print(f"🗑️  Deleted HPO study: {study_name}")
        else:
            print(f"❌ HPO study not found: {study_name}")

## Checkpoint Utilities

Utility functions for checkpoint management and visualization.

In [ ]:
# Initialize checkpoint managers
checkpoint_manager = EnhancedCheckpointManager()
auto_resume_manager = AutoResumeManager(checkpoint_manager)
hpo_checkpoint_manager = HPOCheckpointManager()

def display_checkpoint_summary():
    """Display a summary of all checkpoints."""
    print("📊 Checkpoint Summary")
    print("=" * 50)
    
    # Training checkpoints
    training_checkpoints = checkpoint_manager.list_checkpoints()
    if not training_checkpoints.empty:
        print("\n🎯 Training Checkpoints:")
        display(training_checkpoints)
    else:
        print("\n🎯 No training checkpoints found")
    
    # HPO studies
    hpo_studies = hpo_checkpoint_manager.list_hpo_studies()
    if not hpo_studies.empty:
        print("\n🔍 HPO Studies:")
        display(hpo_studies)
    else:
        print("\n🔍 No HPO studies found")

def cleanup_old_checkpoints(keep_best_n: int = 3):
    """Clean up old checkpoints, keeping only the best N."""
    checkpoints_df = checkpoint_manager.list_checkpoints()
    
    if checkpoints_df.empty:
        print("No checkpoints to clean up")
        return
    
    # Group by experiment and keep best N for each
    experiments = checkpoints_df['experiment'].unique()
    
    for experiment in experiments:
        exp_checkpoints = checkpoints_df[checkpoints_df['experiment'] == experiment]
        exp_checkpoints = exp_checkpoints.sort_values('best_metric', ascending=False)
        
        if len(exp_checkpoints) > keep_best_n:
            to_delete = exp_checkpoints.iloc[keep_best_n:]
            
            print(f"\n🧹 Cleaning up experiment: {experiment}")
            for _, checkpoint in to_delete.iterrows():
                checkpoint_manager.delete_checkpoint(checkpoint['checkpoint_id'])

def export_checkpoint_info(checkpoint_id: str) -> Dict[str, Any]:
    """Export detailed checkpoint information."""
    try:
        info = checkpoint_manager.get_checkpoint_info(checkpoint_id)
        print(f"📋 Checkpoint Information: {checkpoint_id}")
        print("=" * 50)
        print(f"Experiment: {info['metadata']['experiment_name']}")
        print(f"Model: {info['metadata']['model_type']}")
        print(f"Epoch: {info['metadata']['epoch']}")
        print(f"Best Metric: {info['metadata']['best_metric']:.4f}")
        print(f"Size: {info['size_mb']:.2f} MB")
        print(f"Files: {', '.join(info['files'])}")
        print(f"Created: {info['metadata']['created_at']}")
        if info['metadata']['notes']:
            print(f"Notes: {info['metadata']['notes']}")
        
        return info
    except FileNotFoundError as e:
        print(f"❌ {e}")
        return {}

# Display initial summary
display_checkpoint_summary()

print("\n✅ Enhanced Checkpoint System Ready!")
print("\nKey Features:")
print("  🔄 Auto-resume training from interruptions")
print("  💾 Complete state saving (model, optimizer, scheduler, metadata)")
print("  🔍 HPO study checkpointing and resumption")
print("  📊 Checkpoint visualization and management")
print("  🧹 Automatic cleanup of old checkpoints")